## Hands-on Example

### First: Import libraries

In [1]:
import requests
import json
from elasticsearch import Elasticsearch
# uncomment the next lines if don't have requests and Elasticsearch libraries on your machine 
# pip install requests
# pip install elasticsearch 

### Second: Connect to the cluster and check its status

In [2]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [3]:
# check the status of the cluster 
# if you just started the cluster, wait a bit before running this command. 
# Otherwise, you will get a "no response" error. If that happens, retry in a few seconds. 
print(es.cluster.health())

{'cluster_name': 'es-docker-cluster', 'status': 'green', 'timed_out': False, 'number_of_nodes': 4, 'number_of_data_nodes': 3, 'active_primary_shards': 0, 'active_shards': 0, 'relocating_shards': 0, 'initializing_shards': 0, 'unassigned_shards': 0, 'delayed_unassigned_shards': 0, 'number_of_pending_tasks': 0, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 0, 'active_shards_percent_as_number': 100.0}


### Third: Create two indices

In [4]:
settings1 = {"number_of_replicas": 0,"number_of_shards": 2}
es.indices.create(index='exp1_quotes', settings=settings1)

settings2 = {"number_of_replicas": 1}
es.indices.create(index='exp2_quotes', settings=settings2)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'exp2_quotes'}

### Fourth: Fetch data from the API and ingest it into both indices

In [5]:
i = 0
r = requests.get('https://programming-quotes-api.herokuapp.com/Quotes?count=501')
quotes = json.loads(r.content)
while i < len(quotes):
    es.index(index='exp1_quotes', document=quotes[i])
    es.index(index='exp2_quotes', document=quotes[i])
    i=i+1 

### Fifth: Investigate data using one of the indices 

In [7]:
res = es.search(index="exp1_quotes", query = {'match_all' : {}}, size = '3')
documents = res['hits']['hits']

for num, doc in enumerate(documents):
    for key, value in doc.items():
        print (key, ":", value)
   
    print ("\n")

_index : exp1_quotes
_type : _doc
_id : BVIMwH0BUrelX8XTSEV0
_score : 1.0
_source : {'id': '5a6ce86d2af929789500e7ca', 'author': 'Edsger W. Dijkstra', 'en': 'The computing scientist’s main challenge is not to get confused by the complexities of his own making.'}


_index : exp1_quotes
_type : _doc
_id : B1IMwH0BUrelX8XTSEWk
_score : 1.0
_source : {'id': '5a6ce86f2af929789500e7f3', 'author': 'Edsger W. Dijkstra', 'en': 'If debugging is the process of removing software bugs, then programming must be the process of putting them in.'}


_index : exp1_quotes
_type : _doc
_id : CVIMwH0BUrelX8XTSEXL
_score : 1.0
_ignored : ['en.keyword']
_source : {'id': '5a6ce86e2af929789500e7d9', 'author': 'Edsger W. Dijkstra', 'en': 'A program is like a poem: you cannot write a poem without writing it. Yet people talk about programming as if it were a production process and measure „programmer productivity“ in terms of „number of lines of code produced“. In so doing they book that number on the wrong side 

## First experiment: Stopping a shard in a system with zero replicas and two shards

In [9]:
### step 2: ingest a document into the second shard
es.index(index='exp1_quotes', document={"id": "1", "author":  "someone", 
                                        "en": "Talk is cheap. Show me the code."})

{'_index': 'exp1_quotes',
 '_type': '_doc',
 '_id': '7FIRwH0BUrelX8XTLkiH',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 1, 'successful': 1, 'failed': 0},
 '_seq_no': 240,
 '_primary_term': 1}

In [11]:
### steps 3 & 5: query the document
res = es.search(index="exp1_quotes", query = {"match" : { "author" : "someone"}})
res

{'took': 132,
 'timed_out': False,
 '_shards': {'total': 2,
  'successful': 1,
  'skipped': 0,
  'failed': 1,
  'failures': [{'shard': 0,
    'index': 'exp1_quotes',
    'node': None,
    'reason': {'type': 'no_shard_available_action_exception',
     'reason': None,
     'index_uuid': 'BCiCzLZYSVSOe8Exz3YuYQ',
     'shard': '0',
     'index': 'exp1_quotes'}}]},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

## Fouth expriment: killing the master node

In [12]:
### step 2: Ingest data into exp2_quotes index 
es.index(index='exp2_quotes', document={"id": "1", "author":  "noone ", 
                                        "en":  "Copy-and-Paste was programmed by programmers for programmers "})

{'_index': 'exp2_quotes',
 '_type': '_doc',
 '_id': '7VIhwH0BUrelX8XT7EgK',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 2, 'failed': 0},
 '_seq_no': 501,
 '_primary_term': 5}